In [1]:
import os
import sys
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from pathlib import Path
import timm
from scipy.signal import butter, sosfiltfilt


from pytorch_tcn import TCN

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..", "..")))


from src.utils.k_folds_creator import KFoldCreator
from src.utils.utils import get_models_save_path
from src.utils.constants import Constants
from src.utils.feature_extraction import FeatureExtraction

2025-11-07 22:52:38,429 :: root :: INFO :: Initialising Utils
2025-11-07 22:52:38,476 :: root :: INFO :: Initialising Datasets
2025-11-07 22:52:38,478 :: root :: INFO :: Initialising Models


In [2]:
class ClassifierHead(nn.Module):
    def __init__(self, tcn_input_size, cnn_input_size, hidden_size, output_size, dropout):
        super().__init__()
        
        # 1. Create a separate BatchNorm for each feature type
        self.tcn_bn = nn.BatchNorm1d(tcn_input_size)
        self.cnn_bn = nn.BatchNorm1d(cnn_input_size)
        # -----------------------------------------------
        
        self.tcn_input_size = tcn_input_size
        self.cnn_input_size = cnn_input_size
        
        total_input_size = tcn_input_size + cnn_input_size
        
        self.fc1 = nn.Linear(total_input_size, hidden_size)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # x is the concatenated tensor: [batch_size, 1792]
        
        # 1. Split the input back into its TCN and CNN parts
        x_tcn, x_cnn = torch.split(x, [self.tcn_input_size, self.cnn_input_size], dim=1)
        
        # 2. Normalize each part *independently*
        x_tcn = self.tcn_bn(x_tcn)
        x_cnn = self.cnn_bn(x_cnn)
        
        # 3. Re-concatenate the (now separately normalized) features
        x = torch.cat([x_tcn, x_cnn], dim=1)
        
        # 4. Proceed with the rest of the model
        x = self.fc1(x); x = self.bn1(x); x = self.relu(x)
        x = self.dropout(x); x = self.fc2(x)
        return x

class FeatureDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
    def __len__(self): return len(self.features)
    def __getitem__(self, idx): return self.features[idx], self.labels[idx]



In [3]:

class CFG:
    # Fold Config
    SEED = 42
    N_SPLITS = 5
    
    # Paths 
    DATA_PATH = '../../../data/'
    TRAIN_CSV_NAME = 'processed_data_sum_votes_window.csv'
    DATA_PREP_VOTE_METHOD = "sum_and_normalize" 
    FEATURE_STORE_PATH = DATA_PATH + 'extracted_feature/'
    
    # TCN Config 
    TCN_MODEL_DIR = "TCNModel"
    TCN_NUM_CHANNELS = 20
    TCN_CHANNEL_SIZES = [64, 128, 128, 256, 256, 512, 512, 512]
    TCN_KERNEL_SIZE = 21
    TCN_DROPOUT = 0.35
    TCN_DOWNSAMPLE_FACTOR = 3
    TCN_EMBED_SIZE = TCN_CHANNEL_SIZES[-1]

    # CNN Config
    CNN_MODEL_DIR = "MultiSpectCNN"
    CNN_MODEL_NAME = 'tf_efficientnet_b3_ns'
    CNN_IN_CHANNELS = 8
    CNN_IMG_SIZE = (128, 256)
    CNN_EEG_SPEC_PATH = '../../../data/custom_eegs/cwt'
    CNN_EMBED_SIZE = 1536
    
    # Feature Head Config 
    HEAD_MODEL_SAVE_DIR = get_models_save_path() / "MultiModalHead" / DATA_PREP_VOTE_METHOD
    HEAD_HIDDEN_SIZE = 256
    HEAD_DROPOUT = 0.4
    HEAD_BATCH_SIZE = 64
    HEAD_EPOCHS = 20
    HEAD_LR = 1e-5
    TARGET_SIZE = 6

    # General Inference
    INFERENCE_BATCH_SIZE = 64
    NUM_WORKERS = 4

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed(CFG.SEED)
CFG.HEAD_MODEL_SAVE_DIR.mkdir(parents=True, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')




# Training

In [4]:
def run_head_training(train_loader, valid_loader, tcn_input_size, cnn_input_size, fold_k):
    """Trains the classifier head for one fold."""
    model = ClassifierHead(
        tcn_input_size=tcn_input_size,
        cnn_input_size=cnn_input_size,
        hidden_size=CFG.HEAD_HIDDEN_SIZE,
        output_size=CFG.TARGET_SIZE,
        dropout=CFG.HEAD_DROPOUT
    ).to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.HEAD_LR)
    loss_fn = nn.KLDivLoss(reduction='batchmean')
    best_val_loss = float('inf')
    
    for epoch in range(CFG.HEAD_EPOCHS):
        model.train()
        for features, labels in train_loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            log_probs = F.log_softmax(outputs, dim=1)
            loss = loss_fn(log_probs, labels)
            loss.backward()
            optimizer.step()
        
        model.eval()
        valid_loss = 0
        with torch.no_grad():
            for features, labels in valid_loader:
                features, labels = features.to(device), labels.to(device)
                outputs = model(features)
                log_probs = F.log_softmax(outputs, dim=1)
                loss = loss_fn(log_probs, labels)
                valid_loss += loss.item() * features.size(0)
        
        valid_loss /= len(valid_loader.dataset)
        
        if valid_loss < best_val_loss:
            best_val_loss = valid_loss
            save_path = CFG.HEAD_MODEL_SAVE_DIR / f'best_head_fold{fold_k}.pth'
            torch.save(model.state_dict(), save_path)
            print(f"Saved Best Model for Fold {fold_k} at Epoch {epoch+1}")
            
        print(f"  Epoch {epoch+1}/{CFG.HEAD_EPOCHS}, Val Loss: {valid_loss:.4f}, Train Loss: {loss.item():.4f}")

    print(f"  Fold {fold_k} Best Val Loss: {best_val_loss:.4f}")
    return best_val_loss




In [5]:
if __name__ == '__main__':
    
    CFG.FEATURE_CACHE_PATH = Path(CFG.FEATURE_STORE_PATH) 
    Path(CFG.FEATURE_CACHE_PATH).mkdir(parents=True, exist_ok=True)
        
    df_path = Path(CFG.DATA_PATH) / CFG.TRAIN_CSV_NAME
    df = pd.read_csv(df_path)

    print("Creating Level 1 folds for OOF feature generation...")
    fold_creator = KFoldCreator(n_splits=CFG.N_SPLITS, seed=CFG.SEED)
    df = fold_creator.create_folds(df, stratify_col='expert_consensus', group_col='patient_id')

    extractor = FeatureExtraction(CFG, device)

    cache_dir = Path(CFG.FEATURE_CACHE_PATH)
    tcn_oof_cache_file = cache_dir / "tcn_features_all_oof.npy"
    cnn_oof_cache_file = cache_dir / "cnn_features_all_oof.npy"
    labels_cache_file = cache_dir / "labels_all.npy"



    if (tcn_oof_cache_file.exists() and 
        cnn_oof_cache_file.exists() and 
        labels_cache_file.exists()):
        
        print("Loading all OOF features from cache...")
        tcn_features_all_oof = np.load(tcn_oof_cache_file)
        cnn_features_all_oof = np.load(cnn_oof_cache_file)
        labels_all = np.load(labels_cache_file)

    else:
        print("Generating OOF features... This will take a while.")
        
        tcn_features_all_oof = np.zeros((len(df), CFG.TCN_EMBED_SIZE), dtype=np.float32)
        cnn_features_all_oof = np.zeros((len(df), CFG.CNN_EMBED_SIZE), dtype=np.float32)
        labels_all = np.zeros((len(df), CFG.TARGET_SIZE), dtype=np.float32)

        for fold_k in range(CFG.N_SPLITS):
            print("\n" + "="*50)
            print(f"Generating OOF features for Fold {fold_k}")
            print("="*50)

            val_indices = df[df['fold'] == fold_k].index
            val_df = df.iloc[val_indices]
            
            eeg_loader_val = extractor.get_eeg_inference_loader(
                val_df, CFG.DATA_PATH, CFG.TCN_DOWNSAMPLE_FACTOR, mode='train'
            )
            spec_loader_val = extractor.get_spec_inference_loader(
                val_df, Constants.TARGETS, CFG.DATA_PATH, CFG.CNN_IMG_SIZE, CFG.CNN_EEG_SPEC_PATH, mode='train'
            )

            print(f"Loading TCN Model for Fold {fold_k}...")
            tcn_model_path = (
                get_models_save_path() / CFG.TCN_MODEL_DIR / 
                CFG.DATA_PREP_VOTE_METHOD / f'best_model_fold{fold_k}.pth'
            )
            tcn_extractor = extractor.build_tcn_feature_extractor(tcn_model_path)
            tcn_features_val, labels_val_tcn = extractor.extract_features(tcn_extractor, eeg_loader_val)
            del tcn_extractor; torch.cuda.empty_cache()

            print(f"Loading CNN Model for Fold {fold_k}...")
            cnn_model_path = (
                get_models_save_path() / CFG.CNN_MODEL_DIR / 
                CFG.DATA_PREP_VOTE_METHOD / f'best_model_fold{fold_k}.pth'
            )
            cnn_extractor = extractor.build_cnn_feature_extractor(cnn_model_path)
            cnn_features_val, labels_val_cnn = extractor.extract_features(cnn_extractor, spec_loader_val)
            del cnn_extractor; torch.cuda.empty_cache()

            if not np.array_equal(labels_val_tcn, labels_val_cnn):
                raise ValueError(f"Label mismatch in OOF generation for Fold {fold_k}!")

            print(f"Storing OOF features for {len(val_indices)} samples...")
            tcn_features_all_oof[val_indices] = tcn_features_val
            cnn_features_all_oof[val_indices] = cnn_features_val
            labels_all[val_indices] = labels_val_tcn

        print("\nOOF feature generation complete. Saving to cache...")
        np.save(tcn_oof_cache_file, tcn_features_all_oof)
        np.save(cnn_oof_cache_file, cnn_features_all_oof)
        np.save(labels_cache_file, labels_all)


    print(f"TCN OOF Features Shape: {tcn_features_all_oof.shape}")
    print(f"CNN OOF Features Shape: {cnn_features_all_oof.shape}")
    print(f"Labels Shape: {labels_all.shape}")

    combined_features_all_oof = np.concatenate([tcn_features_all_oof, cnn_features_all_oof], axis=1)
    input_size = combined_features_all_oof.shape[1]
    print(f"Combined OOF Features Shape: {combined_features_all_oof.shape}")

    
    print("\nCreating new L2 folds for stacking head training...")
    L2_SEED = CFG.SEED + 42 
    
    head_fold_creator = KFoldCreator(n_splits=CFG.N_SPLITS, seed=L2_SEED)

    df = head_fold_creator.create_folds(df, stratify_col='expert_consensus', group_col='patient_id')
    
    print("New L2 folds created successfully.")


    print("\nStarting 5-Fold Cross-Validation for Classifier Head...")
    all_fold_scores = []

    for fold_k in range(CFG.N_SPLITS):
        print("\n" + "="*50)
        print(f"Training Head for Fold {fold_k}")
        print("="*50)
        
        train_indices = df[df['fold'] != fold_k].index
        val_indices = df[df['fold'] == fold_k].index
        
        train_features = combined_features_all_oof[train_indices]
        train_labels = labels_all[train_indices]
        val_features = combined_features_all_oof[val_indices]
        val_labels = labels_all[val_indices]
        
        print(f"Train features shape: {train_features.shape}")
        print(f"Valid features shape: {val_features.shape}")

        train_dataset = FeatureDataset(train_features, train_labels)
        valid_dataset = FeatureDataset(val_features, val_labels)
        
        train_loader = DataLoader(
            train_dataset, batch_size=CFG.HEAD_BATCH_SIZE, shuffle=True,
            num_workers=CFG.NUM_WORKERS, pin_memory=True
        )
        valid_loader = DataLoader(
            valid_dataset, batch_size=CFG.HEAD_BATCH_SIZE, shuffle=False,
            num_workers=CFG.NUM_WORKERS, pin_memory=True
        )
        
        print(f"Training classifier head for Fold {fold_k}...")
        best_fold_loss = run_head_training(
            train_loader, valid_loader, CFG.TCN_EMBED_SIZE, CFG.CNN_EMBED_SIZE, fold_k
        )
        all_fold_scores.append(best_fold_loss)


    print("\n" + "="*50)
    print("Full 5-Fold Cross-Validation Complete.")
    print(f"Scores per fold: {all_fold_scores}")
    
    mean_cv_score = np.mean(all_fold_scores)
    print(f"\nMean CV Score: {mean_cv_score:.4f}")
    print("="*50)

Creating Level 1 folds for OOF feature generation...
Generating OOF features... This will take a while.

Generating OOF features for Fold 0
Loading TCN Model for Fold 0...


Extracting Features:   0%|          | 0/53 [00:00<?, ?it/s]

Loading CNN Model for Fold 0...


/home/maiko/miniconda3/envs/aicomp/lib/python3.13/site-packages/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(


Extracting Features:   0%|          | 0/53 [00:00<?, ?it/s]

...

Saved Best Model for Fold 3 at Epoch 12
  Epoch 12/20, Val Loss: 0.5430, Train Loss: 0.4773


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8bed65cfe0>
Traceback (most recent call last):
  File "/home/maiko/miniconda3/envs/aicomp/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/home/maiko/miniconda3/envs/aicomp/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
  File "/home/maiko/miniconda3/envs/aicomp/lib/python3.13/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8bed65cfe0>
Traceback (most recent call last):
  File "/home/maiko/miniconda3/envs/aicomp/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/home/maiko/miniconda3/envs/aicomp/lib/py

Saved Best Model for Fold 3 at Epoch 13
  Epoch 13/20, Val Loss: 0.5380, Train Loss: 0.4601


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8bed65cfe0>
Traceback (most recent call last):
  File "/home/maiko/miniconda3/envs/aicomp/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/home/maiko/miniconda3/envs/aicomp/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
  File "/home/maiko/miniconda3/envs/aicomp/lib/python3.13/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f8bed65cfe0>
Traceback (most recent call last):
  File "/home/maiko/miniconda3/envs/aicomp/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/home/maiko/miniconda3/envs/aicomp/lib/py

  Epoch 14/20, Val Loss: 0.5385, Train Loss: 0.4657
Saved Best Model for Fold 3 at Epoch 15
  Epoch 15/20, Val Loss: 0.5328, Train Loss: 0.4665
  Epoch 16/20, Val Loss: 0.5334, Train Loss: 0.4687
Saved Best Model for Fold 3 at Epoch 17
  Epoch 17/20, Val Loss: 0.5301, Train Loss: 0.4691
  Epoch 18/20, Val Loss: 0.5326, Train Loss: 0.4692
  Epoch 19/20, Val Loss: 0.5329, Train Loss: 0.4737
Saved Best Model for Fold 3 at Epoch 20
  Epoch 20/20, Val Loss: 0.5275, Train Loss: 0.4693
  Fold 3 Best Val Loss: 0.5275

Training Head for Fold 4
Train features shape: (13812, 2048)
Valid features shape: (3277, 2048)
Training classifier head for Fold 4...
Saved Best Model for Fold 4 at Epoch 1
  Epoch 1/20, Val Loss: 0.8433, Train Loss: 1.0524
Saved Best Model for Fold 4 at Epoch 2
  Epoch 2/20, Val Loss: 0.7058, Train Loss: 0.9211
Saved Best Model for Fold 4 at Epoch 3
  Epoch 3/20, Val Loss: 0.6491, Train Loss: 0.8477
Saved Best Model for Fold 4 at Epoch 4
  Epoch 4/20, Val Loss: 0.6171, Train Lo